In [1]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric

In [2]:
squad_dataset = load_dataset('squad')

Reusing dataset squad (/home/erolm_a/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7)


In [6]:
squad_dataset["train"]["answers"]

swer_start': [54], 'text': ['Blue Ivy Carter']},
 {'answer_start': [74], 'text': ['Lenox Hill Hospital']},
 {'answer_start': [160], 'text': ['Glory']},
 {'answer_start': [54], 'text': ['Blue Ivy Carter']},
 {'answer_start': [457], 'text': ['B.I.C.']},
 {'answer_start': [3], 'text': ['January 7, 2012']},
 {'answer_start': [54], 'text': ['Blue Ivy Carter']},
 {'answer_start': [160], 'text': ['Glory']},
 {'answer_start': [367], 'text': ["Blue Ivy's cries"]},
 {'answer_start': [457], 'text': ['B.I.C.']},
 {'answer_start': [880], 'text': ['George Zimmerman']},
 {'answer_start': [112], 'text': ['America the Beautiful']},
 {'answer_start': [398], 'text': ['4 million']},
 {'answer_start': [700], 'text': ['same sex marriage']},
 {'answer_start': [840], 'text': ['a rally']},
 {'answer_start': [112], 'text': ['America the Beautiful']},
 {'answer_start': [112], 'text': ['America the Beautiful']},
 {'answer_start': [186], 'text': ['At Last']},
 {'answer_start': [458], 'text': ['Tumblr']},
 {'answer

In [7]:
squad_dataset.map(lambda x: {"length": len(x["context"])})

Loading cached processed dataset at /home/erolm_a/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7/cache-e31338153c845b21.arrow
Loading cached processed dataset at /home/erolm_a/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7/cache-3d1f03e4f978c026.arrow


DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'length', 'question', 'title'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'length', 'question', 'title'],
        num_rows: 10570
    })
})

In [8]:
squad_dataset["train"]["answers"][:5]

[{'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 {'answer_start': [188], 'text': ['a copper statue of Christ']},
 {'answer_start': [279], 'text': ['the Main Building']},
 {'answer_start': [381], 'text': ['a Marian place of prayer and reflection']},
 {'answer_start': [92], 'text': ['a golden statue of the Virgin Mary']}]

In [9]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]''answer_end'])

assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])



INFO:filelock:Lock 140261613437904 acquired on /home/erolm_a/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 579kB/s]
INFO:filelock:Lock 140261613437904 released on /home/erolm_a/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


In [10]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# what happens with 2?

tokenized_text[masked_index+2] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', '[MASK]', 'puppet', '##eer', '[SEP]']
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors, output_hidden_states=True)
    
    for hid in outputs.hidden_states:
        print(hid.size())
    predictions = outputs.logits[0]

In [ ]:
predictions

In [28]:
predicted_index_1 = torch.argmax(predictions[0, masked_index]).item()
predicted_index_2 = torch.argmax(predictions[0, masked_index+2]).item()

print(tokenizer.convert_ids_to_tokens([predicted_index_1, predicted_index_2]))

['henson', 'a']


In [1]:
import datasets

squad_metric, squad_v2_metric = datasets.load_metric('squad'), datasets.load_metric('squad_v2')

In [2]:
squad_metric

Metric(name: "squad", features: {'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}}, usage: """
Computes SQuAD scores (F1 and EM).
Args:
    predictions: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair as given in the references (see below)
        - 'prediction_text': the text of the answer
    references: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair (see above),
        - 'answers': a Dict in the SQuAD dataset format
            {
                'text': list of possible texts for the answer, as a list of strings
                'answer_start': list of start positions for the answer, as a list of ints
   

In [3]:
squad_v2_metric

Metric(name: "squad_v2", features: {'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', id=None), 'no_answer_probability': Value(dtype='float32', id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}}, usage: """
Computes SQuAD v2 scores (F1 and EM).
Args:
    predictions: List of triple for question-answers to score with the following elements:
        - the question-answer 'id' field as given in the references (see below)
        - the text of the answer
        - the probability that the question has no answer
    references: List of question-answers dictionaries with the following key-values:
            - 'id': id of the question-answer pair (see above),
            - 'answers': a list of Dict {'text': text of the answer as a string}
    no_answer_threshold: float
        Probability threshold

In [4]:
print(squad_metric.description)


This metric wrap the official scoring script for version 1 of the Stanford Question Answering Dataset (SQuAD).

Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by
crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span,
from the corresponding reading passage, or the question might be unanswerable.



In [5]:
import datasets
from tools.dataloaders import SQuADDataloader
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
np.random.seed(42)

squad_dataset = SQuADDataloader()


def squad_collate_fn(rows):
    keys = rows[0].keys()
    return {key: [row[key] for row in rows] for key in keys}

squad_train_dataset = squad_dataset.train_dataset

FULL_FINETUNING=False
if not FULL_FINETUNING:
    squad_dev_size = int(0.01*len(squad_dataset.train_dataset))
    squad_dev_indices = np.random.choice(len(squad_dataset.train_dataset), size=squad_dev_size)
    squad_train_sampler = SubsetRandomSampler(squad_dev_indices, generator=torch.Generator().manual_seed(42))
    squad_train_dataloader = DataLoader(squad_train_dataset, sampler=squad_train_sampler, batch_size=8, collate_fn=squad_collate_fn)

else:
    squad_train_dataloader = DataLoader(squad_train_dataset, batch_size=8, collate_fn=squad_collate_fn)

squad_validation_dataset = squad_dataset.validation_dataset
squad_validation_dataloader = DataLoader(squad_validation_dataset, batch_size=8, collate_fn=squad_collate_fn)

Reusing dataset squad (/home/erolm_a/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7)
100%|██████████| 11/11 [00:09<00:00,  1.18ba/s]


In [6]:
from transformers import BertForQuestionAnswering, BertModel
from torch.nn import Module

bert_normal = BertModel.from_pretrained('bert-base-uncased')
config = bert_normal.config
config.num_labels = 1024
model_qa = BertForQuestionAnswering.from_pretrained('bert-base-uncased', config=config)

class BQAShim(Module):
    def __init__(self, model_qa):
        super().__init__()
        self.model_qa = model_qa
    
    def forward(self, input_ids, attention_mask, token_type_ids, start_positions=None, end_positions=None):
        return self.model_qa(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, start_positions=start_positions, end_positions=end_positions)

model_qa_shimmed = BQAShim(model_qa)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [38]:
from models.training import train_model, get_optimizer, get_schedule
import torch

def parse_batch(batch):
    input_ids = torch.tensor(batch['input_ids'])
    attention_mask = torch.FloatTensor(batch['attention_mask'])
    token_type_ids = torch.tensor(batch['token_type_ids'])
    start = torch.tensor(batch['answer_start'])
    end = torch.tensor(batch['answer_end'])
    
    return (input_ids, attention_mask, token_type_ids, start, end), (batch,)


def my_metric(inputs, outputs):
    """
        predictions: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair as given in the references (see below)
        - 'prediction_text': the text of the answer
    references: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair (see above),
        - 'answers': a Dict in the SQuAD dataset format
            {
                'text': list of possible texts for the answer, as a list of strings
                'answer_start': list of start positions for the answer, as a list of ints
            }
            Note that answer_start values are not taken into account to compute the metric.
    """
    batch_input = inputs[-1]
    references = batch_input['answers']
    # outputs = total_loss, answer_start_logits, answer_end_logits
    answer_start_logits = outputs[1].detach().cpu()
    answer_end_logits = outputs[1].detach().cpu()
    
    answer_starts = torch.argmax(answer_start_logits, 1)
    answer_ends = torch.argmax(answer_end_logits, 1)

    prediction_texts = squad_dataset.reconstruct_sentence(inputs[0], answer_starts, answer_ends)

    predictions = [{
        "id": id,
        "prediction_text": prediction_text
    } for id, prediction_text in zip(references["id"], prediction_texts)]


    squad_metric.add_batch(predictions, references)


optimizer = get_optimizer(model_qa)
scheduler = get_schedule(1, optimizer, squad_train_dataloader)
train_model(model_qa_shimmed, squad_train_dataloader, squad_validation_dataloader, parse_batch, optimizer, scheduler, 1, my_metric)

  0%|          | 0/10950 [00:00<?, ?it/s]8
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8])
  0%|          | 0/10950 [00:13<?, ?it/s]


KeyboardInterrupt: 